In [ ]:
from functools import cache
from os import error
from typing import Counter
from matplotlib.dates import relativedelta
from narwhals import DataFrame
import pandas as pd
from assetAllocation import AssetAllocation
import commonHelper
from dataDownloader import DataDownloader
from db_financialStatement import DB_FinancialStatement
from db_stock import DB_Stock
from db_nyse import DB_NYSE
from commonHelper import EDateType, EFinancialStatementType
from datetime import datetime
import yfinance as yf
from portfolio import Portfolio
import numpy as np
from scipy.stats import norm
from dateutil.parser import parse
import itertools
from typing import Callable

import portfolio

# 전체 컬럼 표시 제한 해제
pd.set_option('display.max_columns', None)

# 추가적으로 전체 행도 보이게 하고 싶다면:
pd.set_option('display.max_rows', None)

# 너무 긴 출력이 잘리는 것도 방지하려면:
# pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)  # 또는 float('inf')


def get_rank_table_from_fs(
    csv_file_name: str,
    loaded: bool,
    filter_fn: Callable[[pd.DataFrame], pd.DataFrame],
    sort_key_fn: Callable[[pd.DataFrame], pd.Series],
    top_n: int = 20
) -> pd.DataFrame:
    """
    공통 랭킹 테이블 생성 함수.

    Parameters:
        csv_file_name: 저장/불러올 CSV 파일명
        loaded: True면 CSV 불러오기, False면 새로 생성
        filter_fn: df -> filtered_df (필터 조건 함수)
        sort_key_fn: df -> pd.Series (정렬 기준 함수)
        top_n: 상위 몇 개 종목만 가져올지
    
    Returns:
        pd.DataFrame: 쿼터별 상위 심볼 테이블
    """

    if loaded:
        result_df = pd.read_csv(csv_file_name)
        result_df = result_df.drop(columns=['Unnamed: 0'], errors='ignore')
        return result_df

    else:
        with DB_FinancialStatement() as fs:
            symbols = fs.get_symbol_list()
            df = fs.get_fs_all(symbols, EDateType.QUARTER)
            df = fs.get_value_data(df)
            df = DB_FinancialStatement.add_quarter_column(df)

            df = filter_fn(df).copy()  # 필터링
            df['__sort_key__'] = sort_key_fn(df)  # 정렬 기준 컬럼 추가

            group_dfs = [group for _, group in df.groupby('Quarter')]

            group_dict = {}
            max_len = 0

            for group in group_dfs:
                quarter = group.iloc[0]['Quarter']
                top_symbols = group.sort_values(by='__sort_key__', ascending=False)['Symbol'].tolist()
                top_symbols = top_symbols[:top_n]
                group_dict[quarter] = top_symbols
                max_len = max(max_len, len(top_symbols))

            # 길이 맞추기 (NaN 채우기)
            for key, value in group_dict.items():
                value.extend([np.nan] * (max_len - len(value)))

            result_df = pd.DataFrame(group_dict)
            result_df.to_csv(csv_file_name, index=False)

            return result_df
        

def get_ncva_rank_table(loaded=True) -> pd.DataFrame:
    return get_rank_table_from_fs(
        csv_file_name='naive_ncva.csv',
        loaded=loaded,
        filter_fn=lambda df: df[(df['LiquidationValue'] > df['MarketCap']) & (df['NetIncome'] > 0)],
        sort_key_fn=lambda df: df['LiquidationValue'] / df['MarketCap'],
        top_n=20
    )

def get_super_value_rank_table(loaded=True) -> pd.DataFrame:
    return get_rank_table_from_fs(
        csv_file_name='super_value.csv',
        loaded=loaded,
        filter_fn=lambda df: df,  # 필터 없음
        sort_key_fn=lambda df: ((1 / df['PER']) + (1 / df['PBR']) + (1 / df['PCR']) + (1 / df['PSR'])) / 4,
        top_n=20
    )


start_date = '2023-07-01'
end_date = '2025-08-04'

df_ncva_rank = get_super_value_rank_table()
symbols = list(set(val for val in df_ncva_rank.values.ravel() if pd.notna(val)))
quarter_list = df_ncva_rank.columns.to_list()

date_dict = commonHelper.get_date_dict_by_quarter_lazy(quarter_list)
date_dict = commonHelper.get_trimmed_date_dict(date_dict, start_date, end_date)
date_dict = commonHelper.adjust_start_data_dict_by_quarter(date_dict, quarter_list[0])
oldest, latest = commonHelper.get_date_range_from_quarters(date_dict)

df = AssetAllocation.get_stock_data_with_ma(symbols=symbols, start_date=oldest, end_date=latest, mas=[10], type='ma_month')
df = AssetAllocation.filter_close_last_month(df)
df = AssetAllocation.strategy_ncva(df, df_ncva_rank, date_dict)
df = AssetAllocation.get_performance(df, 'Super Value')
display(df)


2022-10-22 00:00:00


,Date,Symbol,Result Close,MMA_10,End Balance,Restart Asset,Restart Balance,Balance
0,2023-11-30,"[ASGN, ZENV, LOMA, MBC, NATL, HEPS, FTRE, KEP,...","[89.24, 1.06, 6.52, 13.42, 22.21, 1.4, 29.44, ...","[79.7, 0.98, 6.48, 10.97, 4.43, 1.29, 17.99, 6...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[KEP, KT, LPL, CIG, SKM, CCU, LOMA, HEPS, ZENV...","[500, 500, 500, 500, 500, 500, 500, 500, 500, ...",10000
1,2023-12-29,"[ASGN, ZENV, LOMA, MBC, NATL, HEPS, FTRE, KEP,...","[96.17, 1.18, 7.09, 14.85, 24.29, 1.8, 34.9, 7...","[80.43, 0.98, 6.51, 11.48, 6.86, 1.38, 21.48, ...","[497.94, 514.55, 501.04, 523.53, 475.56, 511.8...","[KEP, KT, LPL, CIG, SKM, CCU, LOMA, HEPS, ZENV...","[497.94, 514.55, 501.04, 523.53, 475.56, 511.8...",10596.29
2,2024-01-31,"[ASGN, ZENV, LOMA, MBC, NATL, HEPS, FTRE, KEP,...","[92.82, 1.14, 7.09, 14.07, 22.39, 1.57, 30.96,...","[81.45, 1.0, 6.53, 12.08, 9.09, 1.4, 24.58, 7....","[507.56, 504.98, 448.03, 514.71, 465.56, 486.1...","[KEP, KT, LPL, CIG, SKM, CCU, LOMA, HEPS, ZENV...","[507.56, 504.98, 448.03, 514.71, 465.56, 486.1...",10105.14
3,2024-02-29,"[ASGN, ZENV, LOMA, MBC, NATL, HEPS, FTRE, KEP,...","[99.32, 2.05, 6.57, 17.31, 21.76, 1.55, 37.54,...","[84.22, 1.11, 6.57, 13.01, 11.27, 1.46, 28.33,...","[640.12, 548.24, 434.52, 535.3, 486.67, 474.7,...","[KEP, KT, LPL, CIG, SKM, CCU, LOMA, HEPS, ZENV...","[640.12, 548.24, 434.52, 535.3, 486.67, 474.7,...",11009.44
4,2024-03-28,"[ASGN, ZENV, LOMA, MBC, NATL, HEPS, FTRE, KEP,...","[104.76, 2.66, 6.68, 18.74, 19.75, 1.5, 40.14,...","[88.16, 1.3, 6.63, 13.84, 13.25, 1.5, 32.35, 7...","[572.12, 536.75, 426.2, 567.65, 479.11, 489.39...","[KEP, KT, LPL, CIG, SKM, CCU, LOMA, HEPS, ZENV...","[572.12, 536.75, 426.2, 567.65, 479.11, 489.39...",11300.47
5,2024-04-30,"[ASGN, ZENV, LOMA, MBC, NATL, HEPS, FTRE, KEP,...","[96.45, 2.1, 7.14, 16.67, 19.93, 1.45, 36.59, ...","[90.24, 1.43, 6.62, 14.35, 15.24, 1.48, 32.61,...","[525.42, 483.92, 419.96, 544.12, 454.0, 493.88...","[RNW, CPBI, SPHR, PHIN, TORO, CLCO, KLG, MBC, ...","[521.0, 521.0, 521.0, 521.0, 521.0, 521.0, 521...",10422.85
6,2024-05-31,"[ASGN, ZENV, LOMA, MBC, NATL, HEPS, FTRE, KEP,...","[93.91, 3.1, 7.46, 16.71, 27.83, 2.01, 25.39, ...","[92.0, 1.63, 6.7, 14.78, 18.02, 1.52, 31.95, 7...","[553.33, 515.83, 489.36, 597.95, 564.73, 620.3...","[RNW, CPBI, SPHR, PHIN, TORO, CLCO, KLG, MBC, ...","[553.33, 515.83, 489.36, 597.95, 564.73, 620.3...",10471.43
7,2024-06-28,"[ASGN, ZENV, LOMA, MBC, NATL, HEPS, FTRE, KEP,...","[88.17, 2.59, 6.76, 14.68, 27.02, 2.21, 23.34,...","[92.6, 1.8, 6.7, 14.97, 20.72, 1.6, 31.53, 7.4...","[576.42, 523.07, 470.05, 525.81, 499.76, 570.4...","[KEP, KT, PKX, SKM, TEO, EDN, CCU, TGS, RNW, H...","[501.0, 501.0, 501.0, 501.0, 501.0, 501.0, 501...",10038.79
8,2024-07-31,"[ASGN, ZENV, LOMA, MBC, NATL, HEPS, FTRE, KEP,...","[94.67, 1.9, 6.46, 18.05, 32.15, 3.1, 27.59, 7...","[93.9, 1.88, 6.75, 15.56, 23.94, 1.77, 31.43, ...","[519.37, 527.04, 495.17, 528.32, 439.33, 504.9...","[KEP, KT, PKX, SKM, TEO, EDN, CCU, TGS, RNW, H...","[519.37, 527.04, 495.17, 528.32, 439.33, 504.9...",10141.21
9,2024-08-30,"[ASGN, ZENV, LOMA, MBC, NATL, HEPS, FTRE, KEP,...","[96.16, 1.73, 7.09, 16.04, 28.61, 2.73, 23.06,...","[95.17, 1.95, 6.89, 16.05, 24.59, 1.93, 30.9, ...","[576.61, 529.93, 482.91, 542.45, 482.99, 589.9...","[KEP, KT, PKX, SKM, TEO, EDN, CCU, TGS, RNW, H...","[576.61, 529.93, 482.91, 542.45, 482.99, 589.9...",10127.4
